# Detecting Human Activities Through Smartphone Sensor - Feature engineering

## Import

In [1]:
import os
from tqdm import tqdm
import math
import pandas as pd
import numpy as np
import scipy.fftpack
from scipy import stats


## Global variables

In [2]:
DATA_LEN=3000
WINDOW_OFFSET=200
WINDOW_SIZE=200
ACTIVITY_NAMES = ["walking", "jogging", "stairs", "sitting", "standing", "typing", "brushing teeth",
                  "eating soup", "eating chips", "eating pasta", "drinking from cup", "eating sandwich",
                  "kicking soccer ball", "playing catch tennis ball", "dribbling basket ball",
                  "writing", "clapping", "folding clothes"]
ACTIVITY_CODES_MAPPING = {'A': 'walking',
                          'B': 'jogging',
                          'C': 'stairs',
                          'D': 'sitting',
                          'E': 'standing',
                          'F': 'typing',
                          'G': 'brushing teeth',
                          'H': 'eating soup',
                          'I': 'eating chips',
                          'J': 'eating pasta',
                          'K': 'drinking from cup',
                          'L': 'eating sandwich',
                          'M': 'kicking soccer ball',
                          'O': 'playing catch tennis ball',
                          'P': 'dribbling basket ball',
                          'Q': 'writing',
                          'R': 'clapping',
                          'S': 'folding clothes'}
PBAR_FORMAT='{desc:12} {percentage:3.0f}%|{bar:27}| [ {n:4d}/{total:4d}, {elapsed}<{remaining}{postfix} ]'

## Features

### Average

Average sensor value over the window

In [3]:
def get_avg(a):
    return a.mean()

### Standard deviation

Standard deviation sensor value over the window

In [4]:
def get_stddev(a):
    return np.std(a)

### Variance

Variance sensor value over the window

In [5]:
def get_var(a):
    return np.var(a)



### Average absolute difference

Average absolute difference over the window and the mean of those values

In [6]:
def get_absoldev(a):
    return np.mean(np.abs(a - np.mean(a)))

### Cosine distance

Cosine distance between pair of sensor values over the window

In [7]:
def get_cos(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


## Correlation

Correlation between pair of sensor values over the window

In [8]:
def get_cor(a, b):
    return np.corrcoef(a,b)[0][1]

### Resultant

Average resultant value, computed by squaring each matching x, y, and z value, summing them, taking the square root, and then averaging these values over the window

In [9]:
def get_resultant(x, y, z):
    resultant = np.sqrt(x**2 + y**2 + z**2)
    average_resultant = np.mean(resultant)
    return average_resultant

### Activity extractor

In [10]:
def calc_activity(arr, data_length):
    f = np.empty([data_length], dtype=arr.dtype)
    for i in range(data_length):
        start = i * WINDOW_OFFSET
        f[i] = arr[start]
    return f

### Binned distribution

A binned distribution, where the range of values in the 10s window (max – min value), divide this range into 10 equal-sized bins, and then record the fraction of values in each bin.

In [11]:
def binned_distribution(data):
    data_min = np.min(data)
    data_max = np.max(data)
    bins = np.linspace(data_min, data_max, 11)  # 11 edges, so 10 bins
    # Use np.histogram to calculate the histogram and the fraction of values in each bin
    hist, _ = np.histogram(data, bins=bins)
    # Normalize the histogram to get the fraction
    bin_fractions = hist / len(data)
    return np.array(bin_fractions)

def calc_bindist(arr, bins, data_length):
    f = np.zeros([data_length, bins])
    for i in range(data_length):
        start = i * WINDOW_OFFSET
        f[i] = binned_distribution(arr[start:start+WINDOW_SIZE])
    return f

### MFCC

MFCCs represent short-term power spectrum of a wave, based on a linear cosine transform of a log power spectrum on a nonlinear mel scale of frequency. There are 13 values per axis over the window


In [12]:
def compute_mfcc(signal, sampling_rate=20, num_mfcc=13, frame_size=WINDOW_SIZE, hop_size=WINDOW_OFFSET, num_mels=40):
    # Step 1: Frame the signal
    num_frames = 1 + (len(signal) - frame_size) // hop_size
    frames = np.zeros((num_frames, frame_size))
    for i in range(num_frames):
        start = i * hop_size
        frames[i] = signal[start:start + frame_size] * np.hamming(frame_size)

    # Step 2: Compute the FFT and Power Spectrum
    n_fft = frame_size
    fft = np.fft.rfft(frames, n=n_fft)
    power_spectrum = (1.0 / n_fft) * (np.abs(fft) ** 2)

    # Step 3: Apply Mel Filter Bank
    mel_filters = np.zeros((num_mels, n_fft // 2 + 1))
    mel_min = 0
    mel_max = 2595 * np.log10(1 + (sampling_rate / 2) / 700)  # Convert max frequency to Mel scale
    mel_points = np.linspace(mel_min, mel_max, num_mels + 2)  # Mel points
    hz_points = 700 * (10 ** (mel_points / 2595) - 1)         # Convert Mel points back to Hz
    bin_points = np.floor((n_fft + 1) * hz_points / sampling_rate).astype(int)  # FFT bin indices

    for m in range(1, num_mels + 1):
        mel_filters[m - 1, bin_points[m - 1]:bin_points[m]] = np.linspace(0, 1, bin_points[m] - bin_points[m - 1])
        mel_filters[m - 1, bin_points[m]:bin_points[m + 1]] = np.linspace(1, 0, bin_points[m + 1] - bin_points[m])

    mel_power = np.dot(power_spectrum, mel_filters.T)
    mel_power = np.where(mel_power == 0, np.finfo(float).eps, mel_power)  # Avoid log(0)

    # Step 4: Log Mel Spectrum
    log_mel_power = np.log(mel_power)

    # Step 5: Discrete Cosine Transform (DCT)
    mfccs = scipy.fftpack.dct(log_mel_power, axis=1, norm='ortho')[:, :num_mfcc]

    mfccs = mfccs.T  # Transpose to get shape (num_mfcc x num_frames)

    mfccs =(mfccs - mfccs.min())/(mfccs.max()-mfccs.min())
    return mfccs

### Utility for windowing

Code which performs windowing function over the time series sensor value

In [13]:
# window and perform opertion
def sliding_window(arr, data_length, op):
    f = np.zeros([data_length])
    for i in range(data_length):
        start = i * WINDOW_OFFSET
        inp=[]
        for a in arr:
            inp.append(a[start:start+WINDOW_SIZE])
        f[i] = op(*inp)
    return f


## Extraction

In [14]:
EXTRACTED_FEATURES = ['ACTIVITY',
                        'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9',
                        'Y0', 'Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6', 'Y7', 'Y8', 'Y9',
                        'Z0', 'Z1', 'Z2', 'Z3', 'Z4', 'Z5', 'Z6', 'Z7', 'Z8', 'Z9',
                        'XAVG', 'YAVG', 'ZAVG',
                        'XABSOLDEV', 'YABSOLDEV', 'ZABSOLDEV',
                        'XSTANDDEV', 'YSTANDDEV', 'ZSTANDDEV',
                        'XVAR', 'YVAR', 'ZVAR',
                        'XMFCC0', 'XMFCC1', 'XMFCC2', 'XMFCC3', 'XMFCC4',
                        'XMFCC5', 'XMFCC6', 'XMFCC7',
                        'XMFCC8', 'XMFCC9', 'XMFCC10', 'XMFCC11', 'XMFCC12',
                        'YMFCC0', 'YMFCC1', 'YMFCC2', 'YMFCC3', 'YMFCC4',
                        'YMFCC5', 'YMFCC6', 'YMFCC7',
                        'YMFCC8', 'YMFCC9', 'YMFCC10', 'YMFCC11', 'YMFCC12',
                        'ZMFCC0', 'ZMFCC1', 'ZMFCC2', 'ZMFCC3', 'ZMFCC4',
                        'ZMFCC5', 'ZMFCC6', 'ZMFCC7',
                        'ZMFCC8', 'ZMFCC9', 'ZMFCC10', 'ZMFCC11', 'ZMFCC12',
                        'XYCOS', 'XZCOS', 'YZCOS',
                        'XYCOR', 'XZCOR', 'YZCOR',
                        'RESULTANT',
                        'PARTICIPANT'] # Categirical: 1600 -1650


In [15]:
def get_activity_code(activity_names):
    cat_len=len(activity_names)
    activity_code={}
    for idx, an in enumerate(activity_names):
        for k in ACTIVITY_CODES_MAPPING.keys() :
            if ACTIVITY_CODES_MAPPING[k]==an:
                key = k
        activity_code[key]=idx

    assert(len(activity_code.keys())== cat_len)

    print(f"{activity_code}")
    print(f"Number of Categories {cat_len}")
    return activity_code, cat_len

def extract_feature(src_path, dst_path):
    if not os.path.exists(dst_path):
        os.makedirs(dst_path)
    filelist_train = sorted([txt for txt in os.listdir(src_path + "/") if txt[-4:] == ".txt"])
    filelist_train


    activity_code, cat_len = get_activity_code(ACTIVITY_NAMES)
    data_length = 1 + ((DATA_LEN * len(ACTIVITY_NAMES)) - WINDOW_SIZE) //WINDOW_OFFSET
    print(f"data length {data_length}")
    column_names = ['participant_id' , 'activity_code' , 'timestamp', 'x', 'y', 'z']
    bak_file_cnt = 0
    for idx, txt in enumerate(tqdm(filelist_train, desc="Extracting features", bar_format=PBAR_FORMAT)):
        df_out = pd.DataFrame()
        df = pd.read_csv(src_path + txt, header = None, names = column_names, comment = ";")
        for k in activity_code.keys():
            df_tmp = df[df['activity_code']==k][0:DATA_LEN]
            if df_tmp.shape[0] !=DATA_LEN:
                df_tmp = pd.read_csv(src_path + '/' + filelist_train[bak_file_cnt],
                                          header = None,
                                          names = column_names,
                                          comment = ";")
                df_tmp = df_tmp[df_tmp['activity_code']==k][0:DATA_LEN]
                bak_file_cnt = bak_file_cnt + 1
            df_out = pd.concat([df_out, df_tmp])
        x = df_out['x'].to_numpy()
        y = df_out['y'].to_numpy()
        z = df_out['z'].to_numpy()
        df_processed = pd.DataFrame()
        # AVG
        f_x = sliding_window([x], data_length, get_avg)
        f_y = sliding_window([y], data_length, get_avg)
        f_z = sliding_window([z], data_length, get_avg)

        df_processed['XAVG'] = f_x
        df_processed['YAVG'] = f_y
        df_processed['ZAVG'] = f_z

        # VAR
        f_x = sliding_window([x], data_length, get_var)
        f_y = sliding_window([y], data_length, get_var)
        f_z = sliding_window([z], data_length, get_var)
        df_processed['XVAR'] = f_x
        df_processed['YVAR'] = f_y
        df_processed['ZVAR'] = f_z


        # STDDEV
        f_x = sliding_window([x], data_length, get_stddev)
        f_y = sliding_window([y], data_length, get_stddev)
        f_z = sliding_window([z], data_length, get_stddev)
        df_processed['XSTANDDEV'] = f_x
        df_processed['YSTANDDEV'] = f_y
        df_processed['ZSTANDDEV'] = f_z

        # ABSOLDEV
        f_x = sliding_window([x], data_length, get_absoldev)
        f_y = sliding_window([y], data_length, get_absoldev)
        f_z = sliding_window([z], data_length, get_absoldev)

        df_processed['XABSOLDEV'] = f_x
        df_processed['YABSOLDEV'] = f_y
        df_processed['ZABSOLDEV'] = f_z

        # COS
        f_xy = sliding_window([x, y], data_length, get_cos)
        f_yz = sliding_window([y, z], data_length, get_cos)
        f_xz = sliding_window([x, z], data_length, get_cos)

        df_processed['XYCOS'] = f_xy
        df_processed['YZCOS'] = f_yz
        df_processed['XZCOS'] = f_xz

        # COR
        f_xy = sliding_window([x, y], data_length, get_cor)
        f_yz = sliding_window([y, z], data_length, get_cor)
        f_xz = sliding_window([x, z], data_length, get_cor)

        df_processed['XYCOR'] = f_xy
        df_processed['YZCOR'] = f_yz
        df_processed['XZCOR'] = f_xz

        # resultant
        res = sliding_window([x, y, z], data_length, get_resultant)

        df_processed['RESULTANT'] = res

        # X0-X9, Y0-Y9, Z0-Z9
        f_x = calc_bindist(x, 10, data_length)
        f_y = calc_bindist(y, 10, data_length)
        f_z = calc_bindist(z, 10, data_length)

        for i in range(10):
            x_n = 'X'+str(i)
            y_n = 'Y'+str(i)
            z_n = 'Z'+str(i)
            df_processed[x_n] = f_x[:,i]
            df_processed[y_n] = f_y[:,i]
            df_processed[z_n] = f_z[:,i]

        f_x = compute_mfcc(x)
        f_y = compute_mfcc(y)
        f_z = compute_mfcc(z)
        for i in range(13):
            x_n = 'XMFCC'+str(i)
            y_n = 'YMFCC'+str(i)
            z_n = 'ZMFCC'+str(i)
            df_processed[x_n] = f_x[i][0:data_length]
            df_processed[y_n] = f_y[i][0:data_length]
            df_processed[z_n] = f_z[i][0:data_length]

        df_processed['PARTICIPANT'] = df_out['participant_id'][0:data_length]
        df_processed['ACTIVITY'] = calc_activity(df_out['activity_code'].to_numpy(), data_length)
        df_processed = df_processed[EXTRACTED_FEATURES]
        df_processed.to_csv(dst_path+txt, encoding='utf-8', index=False, header=False)



### Feature extraction for phone accelerometer

In [16]:
src_path="../dataset/raw/phone/accel/"
dst_path="../dataset/processed/phone/accel/"
extract_feature(src_path, dst_path)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'O': 13, 'P': 14, 'Q': 15, 'R': 16, 'S': 17}
Number of Categories 18
data length 270


Extracting features 100%|███████████████████████████| [   51/  51, 00:17<00:00 ]


### Feature extraction for phone gyroscope

In [17]:
src_path="../dataset/raw/phone/gyro/"
dst_path="../dataset/processed/phone/gyro/"
extract_feature(src_path, dst_path)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'O': 13, 'P': 14, 'Q': 15, 'R': 16, 'S': 17}
Number of Categories 18
data length 270


Extracting features 100%|███████████████████████████| [   51/  51, 00:16<00:00 ]


### Feature extraction for watch accelerometer

In [18]:
src_path="../dataset/raw/watch/accel/"
dst_path="../dataset/processed/watch/accel/"
extract_feature(src_path, dst_path)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'O': 13, 'P': 14, 'Q': 15, 'R': 16, 'S': 17}
Number of Categories 18
data length 270


Extracting features 100%|███████████████████████████| [   51/  51, 00:16<00:00 ]


### Feature extraction for watch gyroscope

In [19]:
src_path="../dataset/raw/watch/gyro/"
dst_path="../dataset/processed/watch/gyro/"
extract_feature(src_path, dst_path)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'O': 13, 'P': 14, 'Q': 15, 'R': 16, 'S': 17}
Number of Categories 18
data length 270


Extracting features 100%|███████████████████████████| [   51/  51, 00:16<00:00 ]
